Copyright 2017 Amazon.com, Inc. or its affiliates. All Rights Reserved.<br /><br />Licensed under the Amazon Software License (the "License"). You may not<br />use this file except in compliance with the License. A copy of the<br />License is located at:<br />   http://aws.amazon.com/asl/<br />or in the "license" file accompanying this file. This file is distributed<br />on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, express<br />or implied. See the License for the specific language governing permissions<br />and limitations under the License.
# Deploy the best performing model to a production hosting endpoint

In [ ]:
region='us-west-2'

In [ ]:
TUNING_JOB_NAME='your-tuning-job-name'

In [ ]:
import os
os.environ["AWS_REGION"] = region

In [ ]:
import smhpolib
smhpo = smhpolib.get_smhpo_client()

### Look up best performing model

In [ ]:
from pprint import pprint
hptj = smhpo.describe_tuning_job(TuningJobName=TUNING_JOB_NAME)
pprint(hptj['BestTrainingJob'])

In [ ]:
best_training_job_name = hptj['BestTrainingJob']['TrainingJobName']
best_objective_value = float(hptj['BestTrainingJob']['FinalTuningJobObjectiveMetric']['Value'])
metric_name = hptj['BestTrainingJob']['FinalTuningJobObjectiveMetric']['MetricName']
print("Best Training Job was %s.\nBest score for %s is %g" % (best_training_job_name, metric_name, best_objective_value))

## Set up hosting for the model
In order to set up hosting, we have to import the model from training to hosting. 

### Import model into hosting

Register the model with hosting. This allows the flexibility of importing models trained elsewhere.

In [ ]:
from sagemaker import get_execution_role
import boto3

client = boto3.client('sagemaker')
role = get_execution_role()

In [ ]:
tj_info = client.describe_training_job(TrainingJobName=best_training_job_name)

In [ ]:
model_name=best_training_job_name + '-model'
print(model_name)

model_data = tj_info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

container = tj_info['AlgorithmSpecification']['TrainingImage']
primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

### Create endpoint configuration

SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [ ]:
from time import gmtime, strftime

endpoint_config_name = 'XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

### Create endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
import time

endpoint_name = 'XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Hosting Endpoint Arn: " + resp['EndpointArn'])
print("Status: " + status)

## Validate the model for use
Finally, the customer can now validate the model for use. They can obtain the endpoint from the client library using the result from previous operations, and generate classifications from the trained model using that endpoint.


### Get some sample data
You might load data from the validation dataset which is at this URI:

In [ ]:
valid_s3 = [chan['DataSource'] for chan in tj_info['InputDataConfig'] if chan['ChannelName']=='validation'][0]['S3DataSource']['S3Uri']

But we're just going to specify it as a string to avoid duplicating logic around dataset formating & parsing.

In [ ]:
# First row from validation dataset
sample_record="0,5,1,4,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,1.8,2.332648709,10,0,-1,0,0,14,1,1,0,1,104,2,0.445982062,0.879049073,0.40620192,3,0.7,0.8,0.4,3,1,8,2,11,3,8,4,2,0,9,0,1,0,1,1,1"
label,payload = sample_record.split(',',maxsplit=1)

### Run prediction

In [ ]:
import json
from itertools import islice
import math
import struct

runtime_client = boto3.client('runtime.sagemaker')

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read()
result = float(result.decode("utf-8"))
print ('Label: ',label,'\nPrediction: ', result)

### Delete Endpoint
Once you are done using the endpoint, you can use the following to delete it. 

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)